# #101 Loadcase Generator
<i>Generates loadcases and envelopes from defined in a spreadsheet.</i>
***

Set the path of the spreadsheet with the loadcases to create:

(an example/template can be found in `DataFiles/11 Loadcases.xlsx`)

In [ ]:
data_file = "DataFiles/11 Loadcases.xlsx" # Path to the Excel file

Connect to the running LUSAS model:

In [ ]:
import pandas as pd
import numpy as np
import sys; sys.path.append('../') # Reference modules in parent directory
from shared.LPI import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("This script will add loadcases to an an existing model, please open a model and run the script again.")

db = lusas.database()

Delete all defined analyses: (optional)

In [ ]:
delete_all_previous = True
if delete_all_previous:
    db.deleteAllAnalyses()
    db.createAnalysisStructural("Analysis 1", False)
    db.deleteLoadsets("all") # remaining post-processing loadsets

Read the excel definitions

In [ ]:
df = pd.read_excel(data_file, sheet_name="Loadcases", usecols=range(0,4))
print(df.head())

Optionally change the state of LUSAS to speed up the script excecution time. Mind that LUSAS UI will not respond to user interaction when in this state!

In [ ]:
speed_up = True

if speed_up:
    # Disable the interface to improve performance
    if lusas.getMajorVersionNumber() >= 22 : lusas.enableUI(False)
    # Create a command batch, such that the operation can be undone, this also improves performance
    db.beginCommandBatch("Create Loadcases", "undoable")

Find base analysis:

In [ ]:
allAnalyses : list[IFAnalysisBaseClass] = db.getAnalyses()
base_analysis = ''
for analysis in allAnalyses:
    if analysis.isBase():
        base_analysis = analysis.getName()
        break
print(f"Base analysis is: {base_analysis}")

We'll create a wrapper function that checks for the existance of a loadcase and increments the name

In [ ]:
def safe_create_loadcase(loadcase_name:str, analysis_name:str) -> 'IFLoadcase':
    if not db.existsLoadset(loadcase_name):
        return db.createLoadcase(loadcase_name, analysis_name)
    else:
        i = 1
        while db.existsLoadset(f"{loadcase_name}({i})"):
            i+=1
        db.createLoadcase(f"{loadcase_name} - {i}", analysis_name)

Loop throgh the rows in the loadcase table creating the loadcases as required

In [ ]:
for i, row in df.iterrows():
    loadcase_name = row['Name']
    loadcase_count = row['Count']
    # Make sure we have a string for comparison, it should be Yes or empty
    add_gravity = row['Gravity'] if isinstance(row['Gravity'], str) else ""
    # For the analysis name, we need a string and if the analysis doesnt exist we'll need to create it before trying to add the loadcase
    analysis_name = row['Analysis']

    if not isinstance(analysis_name, str):
        analysis_name = base_analysis
    else:
        if not db.existsAnalysis(analysis_name):
            db.createAnalysisStructural(analysis_name, False)
            print(f"Created analysis {analysis_name}")

    # If the number of requested "similar" loadcases is less than two, create a single loadcase
    if np.isnan(loadcase_count) or int(loadcase_count) < 2:
        loadcase = safe_create_loadcase(loadcase_name, analysis_name)
        print(f"Created loadcase {loadcase.getName()}")
        if add_gravity == "Yes": loadcase.addGravity(True)
    else:
        # Multiple loadcases
        for i in range(1, int(loadcase_count)+1):
            loadcase = safe_create_loadcase(f"{loadcase_name} {i}", analysis_name)
            print(f"Created loadcase {loadcase.getName()}")
            if add_gravity == "Yes": loadcase.addGravity(True)


In [ ]:
def safe_create_envelope(envelope_name:str) -> 'IFEnvelope':
    if not db.existsLoadset(envelope_name):
        return db.createEnvelope(envelope_name)
    else:
        i = 1
        while db.existsLoadset(f"{envelope_name}({i})"):
            i+=1
        return db.createEnvelope(f"{envelope_name} - {i}")

Read in the envelope definition sheets and print the top for rows to confirm data as expected

In [ ]:
df = pd.read_excel(data_file, sheet_name="Envelopes", usecols=range(0,4))
print(df.head())

In [ ]:
for name in df['Name'].unique():
    #print(name)
    df_env = df[df['Name'] == name]
    #print(df_env)
    # Loadset ids to add to the envelope
    ids = []         
    for i, row in df_env.iterrows():
        loadcase_name = row['Loadcases']
        find_similar = row['FindSimilar']
        # Add all lodsets that start with the given name
        if find_similar == "Yes":
            for loadset in db.getLoadsets("Loadcase"):
                if str(loadset.getName()).startswith(loadcase_name):
                    ids.append(loadset.getID())
        else:
            if db.existsLoadset(loadcase_name):
                loadset = db.getLoadset(loadcase_name)
                type_code = loadset.getTypeCode()
                # append the id
                ids.append(loadset.getID())
                if type_code == 3:
                    ids.append(loadset.getAssocLoadset().getID())
                elif type_code == 6:
                    ids.append(loadset.getAssocLoadset().getID())

    
    env = safe_create_envelope(name)
    for id in ids:
        env.addEntry(id)

    # Set the treeview folder name if available
    folder_name = row['Folder']
    if isinstance(folder_name, str) and len(folder_name) > 0:
        env.setTreeLocation(folder_name, True)


Revert the modeller status (changed for script speed up):

In [ ]:
if speed_up:
    # Re-enable the interface. If something goes wrong above this cell must be manually run
    if lusas.getMajorVersionNumber() >= 22 : lusas.enableUI(True)
    # Close the command batch. Note if an error occurs above this cel must be run to restore the interactive user interface
    db.closeCommandBatch()